In [ ]:
#install elevenlabs
!pip install dlib numpy opencv-python


In [ ]:
import time
from collections import deque
import os
from elevenlabs import ElevenLabs, play, Voice, VoiceSettings
import cv2
import dlib

In [ ]:
#Retrieving api key and creating messages based on severity and amount of chimes
import os
from elevenlabs import ElevenLabs, play, Voice, VoiceSettings

class VoiceAlertManager:
    def __init__(self):
        api_key = os.getenv("ELEVENLABS_API_KEY")
        self.client = ElevenLabs(api_key=api_key)
        self.voice = Voice(
            voice_id='YOUR_VOICE_ID',  # Replace with your chosen voice ID
            settings=VoiceSettings(
                stability=0.8,
                similarity_boost=0.6,
                style=0.2,
                use_speaker_boost=True
            )
        )

    def generate_alert(self, chime_count):
        if chime_count >= 5:
            message = "You are getting sleepy. Please pull over immediately."
        elif chime_count >= 3:
            message = "You should rest. You are showing symptoms of sleepiness."
        else:
            message = "Please stay alert."

        audio = self.client.generate(
            text=message,
            voice=self.voice
        )
        play(audio)


In [ ]:
#Main loop cycling through for true conditons
# Initialize components
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
chime_manager = ChimeManager()
voice_alert_manager = VoiceAlertManager()

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    face_detected = len(faces) > 0
    chime_manager.update_head_tilt(face_detected)

    for face in faces:
        landmarks = predictor(gray, face)

        # Implement your yawn, blink, and eyes closed detection logic here
        # For example:
        # if yawn_detected:
        #     chime_manager.update_yawn()
        # if blink_detected:
        #     chime_manager.update_blink()
        # if eyes_closed:
        #     chime_manager.update_eyes_closed(True)
        # else:
        #     chime_manager.update_eyes_closed(False)

    # Generate voice alert based on chime count
    voice_alert_manager.generate_alert(chime_manager.chime_count)

    cv2.imshow("Drowsiness Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()